# Run batch simulation with RE-EMISSION

### Produces a range of GHG estimates for different combinations of parameters

#### T.Janus 14/10/2023

In [ ]:
from typing import Any, Tuple, Dict
from itertools import product
import os
import re
try:
    import reemission
except ModuleNotFoundError:
    !pip install reemission@git+https://github.com/tomjanus/reemission
    import reemission
import subprocess
import json
from reemission.input import Inputs
from reemission.model import EmissionModel

### Define utility functions

In [ ]:
def access_nested_dict(nested_dict: Dict, keys: Tuple[Any, ...]) -> Dict | Any:
    """Returns a value (can be any type including sub (nested) directory) from a nested dictionary for
    a number of nested keys provided in a typle `keys`"""
    try:
        for key in keys:
            nested_dict = nested_dict[key]
        return nested_dict
    except (KeyError, TypeError) as e:
        return None
    

def modify_dict_fields(data: Dict, field_name: Tuple[str, ...], field_value: Any) -> None:
    """ """
    for key, key_data in data.items():
        data_slice = access_nested_dict(key_data, field_name[:-1])
        data_slice[field_name[-1]] = field_value

        
def load_json(json_file: str) -> Dict:
    """ """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return data  


def save_to_json(data: Dict, file_name: str) -> None:
    """ """
    with open(file_name, "w") as file:
        json.dump(data, file, indent=4)

In [ ]:
# Config dictionary for running simulations with combinations (combinatorial) of different options
batch_simu_config = (
    {   "field": ("catchment", "biogenic_factors", "soil_type"),
        "values": ("mineral", "organic"),
        "acronyms": ("MIN", "ORG"),
    },
    {   "field": ("catchment", "biogenic_factors", "landuse_intensity"),
        "values": ("low intensity", "high intensity"),
        "acronyms": ("LOW", "HIGH")
    },
    {   "field": ("catchment", "biogenic_factors", "treatment_factor"),
        "values": (
            "no treatment", "primary (mechanical)", "secondary biological treatment", 
            "tertiary"),
        "acronyms": ("NO", "PRIM", "SEC", "TER")
    },
)

## 1. Create a set of input files

In [ ]:
input_directory = os.path.join('inputs/reemission')

In [ ]:
# Set combinations
value_combination_combo = []
acronym_combination_combo = []
# Generate all combinations of values and acronyms
value_combinations = product(*(item["values"] for item in batch_simu_config))
acronym_combinations = product(*(item["acronyms"] for item in batch_simu_config))

for combo in acronym_combinations:
    acronym_combination_combo.append(combo)
for combo in value_combinations:
    value_combination_combo.append(combo)

input_json_file = os.path.join(input_directory, "reemission_inputs.json")
for value_combo, acronym_combo in zip(value_combination_combo, acronym_combination_combo):
    file_suffix = "_".join(acronym_combo)
    # Load a json file
    input_data: Dict = load_json(input_json_file)
    for ix, item in enumerate(batch_simu_config):
        modify_dict_fields(input_data, item['field'], value_combo[ix])
    # Create a new file name
    base_name, extension = os.path.splitext(input_json_file)
    new_file_name = f"{base_name}_{file_suffix}{extension}"
    print(f"Saving file: {new_file_name}")
    save_to_json(input_data, os.path.join(new_file_name))

## 2. Discover input files in a folder and run simulation for each input file

In [ ]:
file_pattern = r'^reemission_inputs_(.+)\.json$'
file_regex = re.compile(file_pattern)
# List to store matching filenames
matching_files = []
suffixes = []
# Iterate through the files in the directory
for root, dirs, files in os.walk(input_directory):
    for file in files:
        match = file_regex.match(file)
        if match:
            suffix = match.group(1)
            matching_files.append(os.path.join(root, file))
            suffixes.append(suffix)

In [ ]:
outputs_folder = os.path.join("outputs", "reemission")
for ix, (file, suffix) in enumerate(zip(matching_files, suffixes)):
    xls_output_file = "outputs_"+suffix+".xlsx"
    json_output_file = "outputs_"+suffix+".json"
    print(f"Running simulation no. {ix+1}")
    cmd = [
        "reemission", "calculate", file, "-o", os.path.join(outputs_folder, xls_output_file), 
        "-o", os.path.join(outputs_folder, json_output_file)]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)